In [1]:
school = "Pitt"

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
with open("links.txt") as f:
    text = f.read()
text = text.replace('"', '')
all_schools = {k[0].strip(): k[1].strip() for k in (school.split('-') for school in text.split('\n'))}
all_schools

{'Boston College': 'https://bceagles.com/index.aspx',
 'Clemson University': 'https://clemsontigers.com/',
 'Duke University': 'https://goduke.com/index.aspx',
 'Florida State University': 'https://seminoles.com/',
 'Georgia Institute of Technology (Georgia Tech)': 'https://ramblinwreck.com/',
 'University of Louisville': 'https://gocards.com/',
 'University of Miami': 'https://miamihurricanes.com/',
 'University of North Carolina at Chapel Hill': 'https://goheels.com/',
 'North Carolina State University': 'https://gopack.com/',
 'University of Notre Dame': 'https://fightingirish.com/',
 'University of Pittsburgh': 'https://pittsburghpanthers.com/index.aspx',
 'Syracuse University': 'https://cuse.com/',
 'University of Virginia': 'https://virginiasports.com/',
 'Virginia Polytechnic Institute and State University (Virginia Tech)': 'https://hokiesports.com/index.aspx',
 'Wake Forest University': 'https://godeacs.com/'}

In [4]:
def get_roster(link):
    data = pd.read_html(link)[2]
    if 'Full Name' in data.columns:
        data = data.rename(columns = {'Full Name': 'Name'})
    if not ('Hometown' in data.columns and 'High School' in data.columns):
        data[['Hometown', 'High School']] = data['Hometown / High School'].str.replace('/ /', '/').str.split('/', n=1, expand=True)
    return data[["Name", "Hometown", "High School"]]

In [5]:
page = BeautifulSoup(requests.get("https://pittsburghpanthers.com/sports/baseball/schedule").content, 'html.parser')
all_sports = [a.get("href").split('/')[2] for a in page.find_all('a') if a.get('href') and 'roster' in a.get('href') and 'sports' in a.get('href') and 'https' not in a.get('href')]

In [6]:
def all_years(sport):
    df = pd.DataFrame()
    for year in range(2010, 2024):
        print(year)
        season = str(year) + '-' + str(year + 1)[-2:]
        link = f"https://pittsburghpanthers.com/sports/{sport}/roster/{year}?view=2"
        try:
            yearly = pd.DataFrame(get_roster(link))
            yearly['Year'] = year
            df = pd.concat([df, yearly])
        except Exception as error:
            print(link, error)
    df['sport'] = sport
    return df

In [7]:
all_players = pd.DataFrame()
# Update Sports for school
for sport in set(all_sports):
    print(sport)
    all_players = pd.concat([all_players, all_years(sport)])

womens-gymnastics
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
baseball
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
mens-basketball
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
swimming-and-diving
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
womens-volleyball
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
track-and-field
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
womens-basketball
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
softball
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
womens-soccer
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
wrestling
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
cross-country
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
football
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021

In [8]:
all_players['Name'].value_counts()
# all_players.head()

Carson Miller        28
Reagan Flannery      18
Sadie Carey-Tharp    18
Michaela Schiele     17
Caroline Rusinski    17
                     ..
Yanna Marrow          1
Jess Eldridge         1
Nava Susi             1
Valerie Ortega        1
Kilian Vallant        1
Name: Name, Length: 1713, dtype: int64

In [9]:
drop_dupes = all_players.drop_duplicates(subset=['Name', 'Hometown', 'High School'])
drop_dupes['Name'].value_counts()

Carson Miller       2
Alyssa Adrian       1
Ryan Winslow        1
Joey Brungo         1
Pat Amara           1
                   ..
Connor McGaffic     1
Alexis Solak        1
Marissa DeMatteo    1
Olivia Gray         1
Kilian Vallant      1
Name: Name, Length: 1713, dtype: int64

In [10]:
drop_dupes.to_csv(f"done/All_Pitt.csv")